Imports

In [22]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
from scipy import ndimage
from skimage.measure import block_reduce


In [ ]:
def block_calc_2D(in_shp, trgt_shp):
    return int(in_shp[0]/trgt_shp[0]), int(in_shp[1]/trgt_shp[1])
    
def plt_fun(arr, title, size=(10,10)):
    plt.figure(figsize=size)
    plt.imshow(arr)
    plt.title(title)
    plt.show()

def normalize255(img):
    return img/np.max(img)*255

def normalize1(img):
    return (img/np.max(img)).astype(np.float)

Load Image

In [5]:
path = r'.\data\Well2_100min_XY1_EGFP_10X.tif'
img = Image.open(path)
img_arr = np.array(img)

path = r'.\data\Well2_100min_XY1_DAPI_10X.tif'
img = Image.open(path)
dapi_arr = np.array(img)

path = r'.\data\Well2_100min_XY1_CY5_10X1.tif'
img = Image.open(path)
cy5_arr = np.array(img)


Subtraction refers to middle section to be subtracted out of calculations from dapi stain

In [ ]:
# PARAMS:
compress_size = (512, 512) # shrink image for faster calcs. DAPI blob requires low resolution
gauss_sigma = 10 # smooth DAPI blob for consistency
sub_thresh = 50 # remove everything below median image brightness
erosion_cycles = 10 # erode middle more for bigger hole

orig_size = dapi_arr.shape
subtraction = cv2.resize(dapi_arr, dsize=compress_size, interpolation=cv2.INTER_CUBIC)
subtraction = ndimage.gaussian_filter(subtraction, gauss_sigma)
subtraction = subtraction/np.max(subtraction)*100
subtraction[subtraction < sub_thresh] = 0
subtraction[subtraction >= sub_thresh] = 1
subtraction = ndimage.binary_dilation(subtraction.astype(np.uint8), iterations=erosion_cycles)
subtraction = 1 - subtraction
subtraction = cv2.resize(subtraction.astype(float), dsize=orig_size, interpolation=cv2.INTER_CUBIC)

plt_fun(subtraction, "subtraction", size=(3,3))
plt_fun(dapi_arr, "original", size=(3,3))

The below section adaptively subtracts out background

In [ ]:
# PARAMS:
sample_area = 100 # length of square to calculate background in
background_percentile = 0.5 # zero out cells below this brightness

s_len = int(sample_area/2)
new_arr = np.zeros_like(img_arr)
for i in range(s_len,img_arr.shape[0], sample_area):
    for j in range(s_len,img_arr.shape[1], sample_area):
        curr_area = img_arr[i-s_len:i+s_len, j-s_len:j+s_len]
        background = np.percentile(curr_area, background_percentile)
        curr_area = curr_area - background
        curr_area[curr_area < 0] = 0
        new_arr[i-s_len:i+s_len, j-s_len:j+s_len] = curr_area
    print(f"Processing {i/img_arr.shape[0]:.2%}", end='\r')

egfp_arr = np.multiply(new_arr, subtraction)

In [ ]:
plt_fun((egfp_arr/np.max(egfp_arr)*255), "egfp_arr", size=(5,5))
plt_fun(img_arr, "base_img", size=(5,5))

Remove low-brightness noise and high-brightness dead cells

In [ ]:
# PARAMS:
low = 6 # Delete noise below this number
high = 30 # Delete (mostly cells) above this number

filtered_arr = np.array(egfp_arr)
filtered_arr[filtered_arr < low] = 0
filtered_arr[filtered_arr > high] = 0

%matplotlib inline

plt_fun(np.array(egfp_arr[3000:3500, 3000:3500]), "orig", size=(10,10))
plt_fun(filtered_arr[3000:3500, 3000:3500], "filtered", size=(10,10))

In [ ]:
# PARAMS
area = (100,100) # Sum score in rectangles with shape

sum_arr = filtered_arr
sum_arr[sum_arr > 0] = 1
area_calc = block_reduce(sum_arr, area, np.sum)

plt_fun(area_calc, "summed", size=(10,10))